In [1]:
import requests
from bs4 import BeautifulSoup
import json
# Run Scrapper first, then analyzer

In [15]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

links = ['https://geography.wisc.edu/people/faculty/', 'https://geog.ufl.edu/faculty/']# add more links

In [22]:


def extract_text_and_links(soup, element):
    # Initialize a list to hold text and links
    content = []
    for child in element.descendants:
        if child.name == 'a':
            href = child.get('href', '')
            text = child.get_text(strip=True)
            if text:  # If the link has text, append it with the URL
                content.append({'text': text, 'url': href})
        elif child.name is None:  # This is a NavigableString, not a tag
            text = child.strip()
            if text:  # If the string is not empty, append it
                content.append({'text': text})
    return content

def categorize_content(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    body = soup.find('body')

    categorized_content = {}

    for container in body.find_all(True, recursive=False):
        container_name = container.name
        if container_name not in categorized_content:
            categorized_content[container_name] = []
        for element in container.find_all(True, recursive=False):
            element_name = element.name
            element_content = extract_text_and_links(soup, element)
            if element_content:  # Only add if there's content to add
                categorized_content[container_name].append({element_name: element_content})

    return categorized_content



In [24]:
for url in links:
    categorized_text = categorize_content(url)
    json_data = json.dumps(categorized_text, ensure_ascii=False, indent=4)
    filename = 'data/extracted_' + url.replace('/', '_') + '.json'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(json_data)

